# Toolhouse with Ollama

Let's set up the Jupyter environment to handle asynchronous programming and enables the display of rich HTML content.

1. **`import nest_asyncio`**: Imports the `nest_asyncio` library, which allows for the smooth handling of asynchronous tasks in environments like Google Colab, where nested event loops might otherwise cause issues.

2. **`nest_asyncio.apply()`**: Applies a patch that enables running asynchronous code without conflicts. This is particularly useful when working with `asyncio` in notebooks, allowing for proper execution of asynchronous tasks.

3. **`from IPython.display import display, HTML`**: Imports functions from IPython’s display module that allow for the rendering of rich media content (like HTML) directly within the notebook. This enables the output of formatted HTML, which can be used for interactive content or visually enriched outputs.

This setup ensures that you can effectively run and display asynchronous code alongside HTML content in the same notebook.

In [1]:
import nest_asyncio

nest_asyncio.apply()

from IPython.display import display, HTML

Ensure you have Ollama installed locally

In [2]:
!which ollama

/usr/local/bin/ollama


To install Ollama: https://ollama.com/download

Install the ollama package from the Python Package Index (PyPI)

In [3]:
%pip install ollama --quiet

Note: you may need to restart the kernel to use updated packages.


Ensure you Ollama is running and Llama 3.1 is available in Ollama

In [4]:
!ollama list

NAME           	ID          	SIZE  	MODIFIED    
llama3.1:latest	f66fc8dc39ea	4.7 GB	6 hours ago	
mistral:latest 	f974a74358d6	4.1 GB	7 hours ago	


To install Mistral on Ollama run the following cell

In [5]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 73b313b5552d... 100% ▕████████████████▏ 1.4 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


Install the Toolhouse package from the Python Package Index (PyPI)

In [6]:
%pip install toolhouse --quiet

Note: you may need to restart the kernel to use updated packages.


Check available tools in your personal ToolHouse "Tool Store" https://app.toolhouse.ai/

insert ToolHouse API key

In [8]:
from getpass import getpass

# Prompt the user to enter the Exa API key securely
TH_API_KEY = getpass("Enter your ToolHouse API key: ")

In [9]:
from toolhouse import Toolhouse

nest_asyncio.apply()

# Initialize Toolhouse
th = Toolhouse(access_token=TH_API_KEY)

# List available tools
available_tools = th.get_tools()
print("Available tools:")
for tool in available_tools:
    print(f"- {tool['function']['name']}")

Available tools:
- exa_web_search
- scraper


Test the toolhouse call with Ollama

In [10]:
# Install required packages
!pip install toolhouse openai --quiet

import nest_asyncio
from openai import OpenAI
from toolhouse import Toolhouse
import sys
from IPython.display import display, HTML

nest_asyncio.apply()

# Initialize the Toolhouse SDK
try:
    th = Toolhouse(access_token='3e76c9b1-427e-4df6-ace8-7b86a03e776b', provider="openai")
    print("Toolhouse SDK initialized successfully.")
except Exception as e:
    print(f"Error initializing Toolhouse SDK: {e}")
    sys.exit(1)

# Print available tools
print("\nAvailable tools in Toolhouse store:")
available_tools = th.get_tools()
for tool in available_tools:
    print(f"- {tool['function']['name']}")
print()

# Initialize the OpenAI client with Ollama's endpoint
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # Required but unused
)
print("Ollama client initialized via OpenAI-compatible API.")

# Define the model to use
MODEL = "llama3.1"  # Changed to llama3.1
print(f"Using model: {MODEL}")

# Define the messages to send to the model
messages = [
    {
        "role": "user",
        "content": "Search the web for current weather in New York using exa_web_search"
    }
]
print(f"Initial message: {messages[0]['content']}")

# Create the completion request
try:
    print("Sending initial completion request...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=available_tools,
    )
    print("Initial completion request successful.")
except Exception as e:
    print(f"Error in initial completion request: {e}")
    sys.exit(1)

# Run the remote tool and get the result
try:
    print("Running tools...")
    tool_run = th.run_tools(response)
    messages += tool_run
    print(f"Tool execution successful. Added {len(tool_run)} new message(s) to the conversation.")
except Exception as e:
    print(f"Error running tools: {e}")
    sys.exit(1)

# Create the final completion request with the updated messages
try:
    print("Sending final completion request...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=available_tools,
    )
    print("Final completion request successful.")
except Exception as e:
    print(f"Error in final completion request: {e}")
    sys.exit(1)

# Print the response with the answer
if response.choices and response.choices[0].message.content:
    print("\nFinal response:")
    display(HTML(f"<pre>{response.choices[0].message.content}</pre>"))
else:
    print("No response content found in the model's reply.")
    print("Response object:", response)

Toolhouse SDK initialized successfully.

Available tools in Toolhouse store:
- exa_web_search
- scraper

Ollama client initialized via OpenAI-compatible API.
Using model: llama3.1
Initial message: Search the web for current weather in New York using exa_web_search
Sending initial completion request...
Initial completion request successful.
Running tools...
Tool execution successful. Added 2 new message(s) to the conversation.
Sending final completion request...
Final completion request successful.

Final response:
